In [1]:
import os

import cv2
from matplotlib import pyplot as plt
import numpy as np

from spatialmath import *
from spatialmath.base import *
from spatialmath.base import sym
from spatialgeometry import *

import glob
import yaml
from configparser import ConfigParser

### Point Cloud Stuff

In [2]:
ply_header = '''ply
format ascii 1.0
element vertex %(vert_num)d
property float x
property float y
property float z
property uchar red
property uchar green
property uchar blue
end_header
'''

def write_ply(fn, verts, colors):
    verts = verts.reshape(-1, 3)
    colors = colors.reshape(-1, 3)
    verts = np.hstack([verts, colors])
    with open(fn, 'wb') as f:
        f.write((ply_header % dict(vert_num=len(verts))).encode('utf-8'))
        np.savetxt(f, verts, fmt='%f %f %f %d %d %d ')


### Stereo Mapper

In [3]:
class StereoMapper():
    def __init__(self) -> None:
        
        config_file = ConfigParser()
        config_file.read("/home/castrou/zed/settings/SN23964822.conf")
        
        # Stereo config
        self.calib_stereo = {}
        for i in config_file['STEREO']:
            self.calib_stereo[i] = config_file.get('STEREO', i)
        # Camera Config
        self.calib_l = {}
        for i in config_file['LEFT_CAM_HD']:
            self.calib_l[i] = config_file.get('LEFT_CAM_HD', i)
        # Camera Config
        self.calib_r = {}
        for i in config_file['RIGHT_CAM_HD']:
            self.calib_r[i] = config_file.get('RIGHT_CAM_HD', i)
            
        # Get stereo settings    
        with open("/home/castrou/zed/settings/zed_oc_stereo.yaml") as stereo_yaml:
            skip_lines = 2
            for i in range(skip_lines):
                _ = stereo_yaml.readline()
            self.stereo_opt = yaml.safe_load(stereo_yaml)
        # Create stereo
        self.stereo = cv2.StereoSGBM_create(
            blockSize=self.stereo_opt['blockSize'],
            minDisparity=self.stereo_opt['minDisparity'], 
            numDisparities=self.stereo_opt['numDisparities'],
            mode=self.stereo_opt['mode'],
            disp12MaxDiff=self.stereo_opt['disp12MaxDiff'],
            preFilterCap=self.stereo_opt['preFilterCap'],
            uniquenessRatio=self.stereo_opt['uniquenessRatio'],
            speckleWindowSize=self.stereo_opt['speckleWindowSize'],
            speckleRange=self.stereo_opt['speckleRange']
        )
        
    def disparity(self, left, right):
        disparity = self.stereo.compute(left, right)  
        disparity = disparity.astype(np.float32) / 16.0
        return disparity
    
    def depth(self, left, right):
        disp = self.disparity(left, right)
        b = self.calib_stereo['Baseline']
        f = self.calib_l['fx']
        Z_p = b*f / disp
        depth = Z_p
        # depth = Z_p.clip(0,30)
        return depth
        
    def pcl(self, left, right, output_name=""):
        print('computing disparity...')
        disp = self.disparity(left, right)
        print('generating 3d point cloud...',)
        h, w = left.shape[:2]
        f = float(self.calib_l['fx'])  # focal length
        Q = np.float32([[1, 0, 0, -0.5*w],
                        [0,-1, 0,  0.5*h], # turn points 180 deg around x-axis,
                        [0, 0, 0,     -f], # so that y-axis looks up
                        [0, 0, 1,      0]])
        points = cv2.reprojectImageTo3D(disp, Q)
        colors = cv2.cvtColor(left, cv2.COLOR_BGR2RGB)
        mask = disp > disp.min()
        out_points = points[mask]
        out_colors = colors[mask]

        write_ply(output_name, out_points, out_colors)
        print('%s saved' % output_name)


### Util

In [4]:
def plot_disparity(disparity, title=''):
    if type(disparity) == list:
        images = disparity
        '''Plot a grid of images'''
        factors = [i for i in range(1, len(images)+1) if len(images) % i == 0]
        ncols = factors[len(factors) // 2] if len(factors) else len(images) // 4 + 1
        nrows = int(len(images) / ncols) + int(len(images) % ncols)
        imgs = [images[i] if len(images) > i else None for i in range(nrows * ncols)]
        f, axes = plt.subplots(nrows, ncols, figsize=(10*ncols, 5*nrows))
        axes = axes.flatten()[:len(imgs)]
        for img, ax in zip(imgs, axes.flatten()): 
            if np.any(img):
                if len(img.shape) > 2 and img.shape[2] == 1:
                    img = img.squeeze()
                ax.imshow(img)
        plt.suptitle(title)
        
    else:
        # plot the disparity map
        plt.figure(figsize=(20,5))
        plt.imshow(disparity,'gray')
        plt.title('Disparity Map using block matching')
        plt.colorbar()
        plt.show()

def get_file_idx(path):
    file = path.split('/')[-1]
    filename = file.split('.')[0]
    idx = filename[5:]
    return int(idx)

### Generate Point Clouds

In [5]:
stereo = StereoMapper()
base_dir = "/home/castrou/university/thesis/data/"

In [6]:
# Load Scan 1
dir = os.path.join(base_dir, "yandiwanba_22-05_1/")
l_dir = os.path.join(dir, "img/left/")
r_dir = os.path.join(dir, "img/right/")
start_idx = 300 # manually discovered
end_idx = 2500 # ""

left_files = [os.path.join(l_dir, file) for file in os.listdir(l_dir) 
                if get_file_idx(file) > start_idx
                and get_file_idx(file) < end_idx]
right_files = [os.path.join(r_dir, file) for file in os.listdir(r_dir) if get_file_idx(file) 
                if get_file_idx(file) > start_idx
                and get_file_idx(file) < end_idx]

for l_img_file, r_img_file in zip(left_files, right_files):
    # idx alignment
    l_idx = get_file_idx(l_img_file)
    r_idx = get_file_idx(r_img_file)
    assert l_idx == r_idx
    idx = l_idx
    
    img_l = cv2.imread(l_img_file)
    img_r = cv2.imread(r_img_file)
    cv2.imshow('left', img_l)
    cv2.imshow('right', img_r)
    pcl = stereo.pcl(img_l, img_r, output_name=os.path.join(dir, f"pcl/cloud{idx}.ply"))
    break

computing disparity...
generating 3d point cloud...
/home/castrou/university/thesis/data/yandiwanba_22-05_1/pcl/cloud2268.ply saved


: 